In [ ]:
from transformers import LongformerTokenizer, LongformerForMaskedLM
import torch

# Load pre-trained model and tokenizer
model_name = 'allenai/longformer-base-4096'
model = LongformerForMaskedLM.from_pretrained(model_name)
tokenizer = LongformerTokenizer.from_pretrained(model_name)

# Function to calculate perplexity
def calculate_perplexity(sentence):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=4096)

    # Move inputs to the appropriate device
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Calculate the loss
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss

    # Calculate perplexity
    perplexity = torch.exp(loss).item()
    return perplexity

# Example sentence
sentence = "The quick brown fox jumps over the lazy dog." * 200  # Making the sentence much longer for demonstration
print("Perplexity:", calculate_perplexity(sentence))

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Perplexity: 1.0005565881729126


In [ ]:
import pandas as pd
import numpy as np

df_with_ucla = pd.read_excel('/content/df_with_ucla.xlsx')
df_without_ucla = pd.read_excel('/content/df_without_ucla.xlsx')

In [ ]:
df_with_ucla_answer = df_without_ucla['question_answer']

In [ ]:
import re

def extract_answers(text):
    # Find all answers in the text using regular expressions
    answers = re.findall(r'Answer:\s*(.*?)(?=Question:|$)', text, re.DOTALL)
    return [answer.strip() for answer in answers]

question_answer = df_with_ucla_answer.values
answers = [extract_answers(t) for t in question_answer]

In [ ]:
perp_list = []
for answer_list in answers:
  perp = [calculate_perplexity(sentence) for sentence in answer_list]
  perp_list.append(perp)

In [ ]:
prep_mean_list = [np.mean(p) for p in perp_list]
prep_mean_list

[2.0794990062713623,
 1.0440348386764526,
 1.6064741611480713,
 1.7743512392044067,
 1.0018887519836426,
 1.013692021369934,
 1.0285018682479858,
 1.0014172792434692,
 2.11081329981486,
 1.7498824993769329,
 1.37009596824646,
 1.9224841594696045,
 1.011672814687093,
 1.0394270022710164,
 1.0642018715540569,
 1.0477749506632488,
 2.1736966133117677,
 1.2849863529205323,
 1.0335652828216553,
 1.071745014190674]

In [ ]:
df = pd.DataFrame(prep_mean_list, columns=['perplexity_values'])

# Specify the file name
csv_file = 'preplexity_without_ucla.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file, index=False)

print(f'List saved to {csv_file}')

List saved to preplexity_without_ucla.csv
